# Collegedata scraping (1)

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

import requests
import time
from bs4 import BeautifulSoup
import re
import TIdatabase as ti

In [2]:
genderdict = {'Male': -1, 'Female': 1}
highschooldict = {'Public': -1, 'Private': 1, 'Parochial': 1, 'Home-Schooled': 1}
minoritylist = ['african', 'hispanic', 'latin','indian', 'native', 'black', 'mexican']
nonminoritylist = ['white', 'caucasian', 'asian', 'european', 'chinese', 'hebrew']
sats=[1600, 1560, 1510, 1460, 1420, 1380, 1340, 1300, 1260, 1220, 1190, 1150, 1110, 1070, 1030, 990, 950, 910, 870, 
      830, 790, 740, 690, 640, 590, 530]
act2satdict=dict(zip(range(11,37),sats))
booleandict={'': -1, 'X': 1}
statusdict={'Will Attend': 1, 'Accepted': 1, 'Applied': 0, 'Deferred': -1, 'Denied': -1, 'Not Applied': 0, 'Wait-Listed': 1, 'Withdrawn': 0, '': 0}

In [3]:
def getFromDict(dictionary,text):
    if dictionary.has_key(text):
        return dictionary[text]
    else:
        return 0
def isMinority(text):
    for m in minoritylist:
        if m in text:
            return 1
    for m in nonminoritylist:
        if m in text:
            return -1
    print text
    return 0

In [15]:
college_ids=['Princeton', 'Harvard', 'Yale', 'Columbia', 'Stanford', 'UChicago', 'MIT', 'Duke', 'UPenn', 'CalTech', 'JohnsHopkins', 'Dartmouth', 'Northwestern', 'Brown', 'Cornell', 'Vanderbilt', 'WashU', 'Rice', 'NotreDame', 'UCB', 'Emory', 'Georgetown', 'CarnegieMellon', 'UCLA', 'USC']
college_urls=[111, 444, 244, 399, 781, 327, 186, 1026, 67, 706, 1509, 403, 1803, 163, 787, 1562, 1720, 731, 1774, 1090, 1039, 1182, 204, 1093, 1138]
college_id_dict=dict(zip(college_ids,college_urls))
uni_list=['Princeton University', 'Harvard College', 'Yale University', 'Columbia University', 'Stanford University', 'University of Chicago', 'Massachusetts Institute of Technology', 'Duke University', 'University of Pennsylvania', 'California Institute of Technology', 'Johns Hopkins University', 'Dartmouth College', 'Northwestern University', 'Brown University', 'Cornell University', 'Vanderbilt University', 'Washington University in St. Louis', 'Rice University', 'University of Notre Dame', 'University of California, Berkeley', 'Emory University', 'Georgetown University', 'Carnegie Mellon University', 'University of California, Los Angeles', 'University of Southern California']
uni_name_dict = dict(zip(uni_list, college_ids))
uni_state=['NJ', 'MA', 'CT', 'NY', 'CA', 'IL', 'MA', 'NC', 'PA', 'CA', 'MD', 'NH', 'IL', 'RI', 'NY', 'TN', 'MO', 'TX', 'IN', 'CA', 'GA', 'DC', 'PA', 'CA', 'CA']
states_dict={'Alabama': 'AL',
 'Alaska': 'AK',
 'American Samoa': 'AS',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Guam': 'GU',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'National': 'NA',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Northern Mariana Islands': 'MP',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virgin Islands': 'VI',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY',
  'Other': 'Other'}
uni_state_dict = dict(zip(uni_list,uni_state))
baseurl='http://www.collegedata.com/cs/admissions/'
tracker_url='admissions_tracker_result.jhtml?schoolId='
student_url='admissions_profile_view.jhtml?profileName='

In [5]:
## Source: https://impythonist.wordpress.com/2015/01/06/ultimate-guide-for-scraping-javascript-rendered-web-pages/
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  
from lxml import html 

class Render(QWebPage):  
    def __init__(self, url):  
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()  
  
    def _loadFinished(self, result):  
        self.frame = self.mainFrame()  
        self.app.quit()

In [6]:
schoolurl=baseurl+tracker_url+str(244)+'&classYear='
r=Render(schoolurl+str(2019))
result = r.frame.toHtml()
#print str(result.toAscii())

In [7]:
soup = BeautifulSoup(str(result.toAscii()),'html.parser')
#r = requests.get(schoolurl+str(2009))
#time.sleep(3)
#soup = BeautifulSoup(r.text,'html.parser')
#print soup.prettify()
#rows = soup.find(id="profilewrap") #.find(id='tableBodyWrap').find_all("tr")[1:]
#print rows

In [8]:
profiles= soup.find_all("a",href=re.compile(r"enterProfileByName"))
profileList=set()
for p in profiles:
    profileList.add(p.get("href").split("'")[1])
print len(profileList)

63


In [5]:
def getScores(scores,fun):
    scores = [s.get_text().strip() for s in scores]
    while '' in scores:
        scores.remove('')
    if len(scores)>0:
        scores=[int(s) for s in scores]
        return fun(scores), len(scores)
    else:
        return None, 0
def getAdmissionTestScore(doc):
    satCRM,dummy = getScores(doc[0:2],sum)
    satW,dummy = getScores([doc[2]],np.max)
    act,dummy = getScores([doc[4]],np.max)
    if act == None:
        return (satCRM+satW)
    elif satCRM==None:
        if satW==None:
            print "Warning: no SAT writing score"
        return act2satdict[act]+satW
    else:
        return (max(act2satdict[act],satCRM)+satW)
def getCanAfford(text):
    if 'Yes' in text:
        return -1
    elif 'No' in text:
        return 1
    else:
        return 0
def removePunct(text):
    text = re.sub(r'([^\s\w]|_)+'," ",text)
    return re.sub('\s+',' ',text).encode('latin-1')

In [6]:
columns_student = ['classrank', 'admissionstest','AP','averageAP','SATsubject', 'GPA', 'GPA_w', 'program','schooltype',
            'intendedgradyear', 'addInfo', 'canAfford', 'female', 'MinorityGender','MinorityRace','international',
           'firstinfamily','sports','artist', 'workexp']
columns_uni = ['collegeID','earlyAppl','visited','alumni', 'outofstate', 'acceptStatus','acceptProb']

In [16]:
def getColumnValues(soup):
    values=dict(zip(columns_student,[None for i in range(len(columns_student))]))
    applications=[]
    # We start with the general information box at the top which includes class year, gender and ethnicity
    doc= soup.find("div",{"class": "general"})
    values['intendedgradyear'] = int(re.findall(r'\d{4}',doc.find("h1").get_text().split('Class of')[-1])[0]) # CLASS YEAR
    doc = doc.find_all("span")
    values['female']= getFromDict(genderdict,doc[0].get_text().strip()) # GENDER
    values['MinorityGender']= 1 if values['female']==0 else -1
    values['MinorityRace'] = isMinority(doc[1].get_text().strip().lower()) # MINORITY RACE
    values['program'] = removePunct(doc[2].get_text().strip()) # PROGRAM
    # Now we look at the academics box which includes GPA and high school info
    doc = soup.find("div", {"class": "academicswrap"}).find_all("span")
    values['schooltype']=getFromDict(highschooldict,doc[0].get_text().strip()) # SCHOOL TYPE
    state=states_dict[doc[1].get_text().strip()] # STATE OF STUDENT
    values['international'] = 1 if state=='Other' else -1
    values['GPA'] = float(doc[3].get_text()) # unweighted GPA
    values['GPA_w'] = float(doc[4].get_text()) if doc[4].get_text().strip()!='' else None # Weighted GPA
    # Next, we go to the test score box which includes SAT, ACT and AP info
    values['admissionstest'] = getAdmissionTestScore(soup.find("div", {"class": "testscorewrap"}).find_all("td")) #Admissions test
    values['SATsubject'] = len(soup.find("caption",text="SAT Subject Test Scores").next_sibling.next_sibling.find_all("tr")) # SAT SUBJECT
    ap_num = len(soup.find("caption",text="AP Examinations").next_sibling.next_sibling.find_all("tr"))
    values['AP']=ap_num #AP
    if ap_num>0:
        doc = soup.find("caption",text="AP Examinations").next_sibling.next_sibling.find_all("td")
        values['averageAP'],values['AP']= getScores(doc,np.mean) # AVERAGE AP
    doc = soup.find_all("div", {"class": "word"})
    doc = [d.get_text().strip() for d in doc]  
    values['addInfo']= removePunct(doc[0]+doc[1]+doc[2]) # Additional info
    # Next: the colleges applied to and the admission results for the admissions table
    doc = soup.find("table", {"class": "collchoice"})
    collegelist = doc.find("tbody").find_all("tr")
    for c in collegelist:
        uni = c.find("th").find("span").get_text().strip()
        if uni in uni_list:
            unirow = dict(zip(columns_uni,[None for i in range(6)]))
            unirow['collegeID']=uni_name_dict[uni]
            doc=c.find_all("td", {"class": "center"})
            unirow['earlyAppl']=booleandict[doc[0].get_text().strip()] # Early Admission
            unirow['alumni']=booleandict[doc[1].get_text().strip()] # Alumni/Legacy
            if values['sports']==None or values['sports']==0: 
                values['sports']=booleandict[doc[2].get_text().strip()] # Athlete
            doc = doc[2].next_sibling.find_next("span")
            unirow['acceptStatus']=statusdict[doc.get_text().strip()] # Admission status
            if values['canAfford']==None or values['canAfford']==0:
                values['canAfford']=getCanAfford(doc.find_next("span").get_text().strip())
            unirow['outofstate']= -1 if state==uni_state_dict[uni] else 1
            applications.append(unirow)
    return values, applications

In [17]:
#if ('students' in locals()): 
#    students.cleanup()
#    del students
#if ('applForm' in locals()): del applForm
#students = ti.Student()
#applForm = ti.ApplForm()
#p=list(profileList)[26]
#p='3q9x4'
#p='meggymack92'
#p='6FNsQ'
#p='pdfrU'
#p='8xnTk'
p='mx4life'
profile_url=baseurl+student_url+p
print profile_url
soup=BeautifulSoup(requests.get(profile_url).text,'html.parser')
#print soup.prettify()
#print columns
newrow, applications = getColumnValues(soup)
#studentID=students.insert(newrow)
#for app in applications:
#    app['studentID']=studentID[0]
#applForm.insert(applications)
#students.save('test_student.csv')
print newrow, applications
#students.df

http://www.collegedata.com/cs/admissions/admissions_profile_view.jhtml?profileName=mx4life
{'intendedgradyear': 2013, 'classrank': None, 'canAfford': 0, 'schooltype': -1, 'artist': None, 'MinorityGender': -1, 'averageAP': 4.25, 'admissionstest': 1900, 'GPA': 3.3, 'GPA_w': 4.3, 'AP': 4, 'program': 'Business administration', 'female': 1, 'workexp': None, 'international': -1, 'addInfo': 'academic team 4 years GK and Math 4 hours week debate team 3 years wasn t established prior 1 5 hours week Drama Club 4 years hours spent varies highly depending on plays etc Tech Crew 4 years hours spent varies highly depending on plays etc Spanish club 3 years 5 hours week SGA Senior Class Treasurer 2 hours every other week Volunteer El Buen Pastor 1 25 hours weekNational Honor Society SGA Senior Class Treasurer Debate Team Treasurer Debate Team Co President Rotary Club Student of the Month AP Scholar with Honorcaptained and lettered in academic team 3rd degree black belt in karate', 'sports': -1, 'Mino

In [84]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applForm' in locals()): del applForm
students=ti.Student()
colleges = ti.College()
applForm = ti.ApplForm()
for p in profileList:
    #print p
    profile_url=baseurl+student_url+p
    soup=BeautifulSoup(requests.get(profile_url).text,'html.parser')
    newrow, applications = getColumnValues(soup)
    studentID=students.insert(newrow)
    for app in applications:
        app['studentID']=studentID[0]
    applForm.insert(applications)
students.df






cambodian

earthling - what else matters?


china

india




studentID classrank  admissionstest  AP averageAP  SATsubject   GPA GPA_w                                       program  schooltype  intendedgradyear                                            addInfo  canAfford  female  MinorityGender  MinorityRace  international firstinfamily  sports artist workexp
0   6DQ8ASVPFX      None            1930   5       4.6           0  3.93  None                        Mechanical Engineering          -1              2019  track and field 15 hours a weekVarsity letter ...          1       1              -1             1             -1          None      -1   None    None
0   NOP4X46KW6      None            2200   0      None           2  3.97  None                                       Biology          -1              2019  BoyScouts NHS NHS world language HOSA Varsity ...         -1      -1              -1             0             -1          None      -1   None    None
0   PF29L59LDN      None            2350   7         5           4  4.00  4.07                                   Mathematics          -1              2019                                                            -1      -1              -1            -1             -1          None      -1   None    None
0   OV1DA5NRBK      None            2360   7         5           3  4.00  4.91                        Industrial Engineering          -1              2019  Varsity Swimming 2 years 60 hours year Nationa...         -1      -1              -1            -1             -1          None      -1   None    None
0   NGVXH37NAJ      None            2250   4       4.5           2  3.89  4.78                                                         0              2019  First Cello in Youth Symphony Writing awards G...         -1       1              -1             1             -1          None      -1   None    None
0   G3XWLEVJ90      None            2170   7  4.571429           3  3.90  4.24          International Relations Biochemistry          -1              2019                                                            -1      -1              -1            -1             -1          None      -1   None    None
0   PYCZ84NJEL      None            2200   0      None           2  4.00   4.4                             Political Science          -1              2019  Long Branch High School Junior Varsity Varsity...         -1       1              -1            -1             -1          None      -1   None    None
0   AZJY8J1585      None            2290   7  4.857143           3  4.00  None                                       geology          -1              2019  Employed as a church organist Select Choir Jaz...          1      -1              -1             1             -1          None      -1   None    None
0   VNY5CL8NZA      None            2110   3  4.333333           3  3.60  None                                    Philosophy          -1              2019  Director of African American Culture Club Gene...         -1      -1              -1             1             -1          None      -1   None    None
0   5KV73WIC45      None            2340   7         5           2  3.98  4.41                                       Biology          -1              2019  Piano for 12 years scholarship pre college stu...         -1       1              -1            -1             -1          None      -1   None    None
0   8MRLU8M8VC      None            2200   2       4.5           0  3.81  4.16                                   Engineering          -1              2019  Run two small businesses 2013 2014 FIRST robot...         -1      -1              -1             0             -1          None      -1   None    None
0   MA0K3TJTBY      None            2140   0      None           0  3.60  4.15                              Computer Science          -1              2019  Varsity swimming 9 12 grades 12 hours wk Robot...         -1      -1              -1             0             -1          None      -1   None    None
0   Z9PKYOQ319

In [85]:
students.save('collegedata_students.csv')
applForm.save('collegedata_applications.csv')
applForm.df

studentID     collegeID  earlyAppl visited  alumni  outofstate  acceptStatus acceptProb
0   6DQ8ASVPFX          Yale         -1    None      -1           1            -1        NaN
1   6DQ8ASVPFX      Stanford         -1    None      -1           1            -1        NaN
2   6DQ8ASVPFX  JohnsHopkins         -1    None      -1           1             1        NaN
0   NOP4X46KW6         UPenn         -1    None      -1           1             1        NaN
1   NOP4X46KW6     Princeton         -1    None      -1          -1             0        NaN
2   NOP4X46KW6         Brown         -1    None      -1           1            -1        NaN
3   NOP4X46KW6          Yale         -1    None      -1           1            -1        NaN
4   NOP4X46KW6       Cornell         -1    None      -1           1            -1        NaN
5   NOP4X46KW6          Duke         -1    None      -1           1            -1        NaN
6   NOP4X46KW6  JohnsHopkins         -1    None      -1           1            -1        NaN
0   PF29L59LDN          Yale         -1    None      -1           1             1        NaN
0   OV1DA5NRBK         UPenn         -1    None      -1           1            -1        NaN
1   OV1DA5NRBK     Princeton         -1    None      -1           1            -1        NaN
2   OV1DA5NRBK           MIT         -1    None      -1           1            -1        NaN
3   OV1DA5NRBK          Yale         -1    None      -1           1            -1        NaN
4   OV1DA5NRBK      UChicago         -1    None      -1           1             1        NaN
5   OV1DA5NRBK     Dartmouth         -1    None      -1           1             1        NaN
6   OV1DA5NRBK       Harvard          1    None      -1           1            -1        NaN
7   OV1DA5NRBK      Stanford         -1    None      -1           1            -1        NaN
8   OV1DA5NRBK       Cornell         -1    None      -1           1            -1        NaN
9   OV1DA5NRBK          Duke         -1    None      -1           1            -1        NaN
10  OV1DA5NRBK  Northwestern         -1    None      -1           1             1        NaN
0   NGVXH37NAJ     Princeton         -1    None      -1           1            -1        NaN
1   NGVXH37NAJ         Brown         -1    None      -1           1             1        NaN
2   NGVXH37NAJ          Yale         -1    None      -1           1            -1        NaN
3   NGVXH37NAJ       Harvard         -1    None      -1           1             1        NaN
0   G3XWLEVJ90          Yale         -1    None      -1           1            -1        NaN
1   G3XWLEVJ90      UChicago          1    None      -1           1             1        NaN
2   G3XWLEVJ90      Columbia          1    None      -1           1            -1        NaN
3   G3XWLEVJ90    Georgetown         -1    None      -1           1            -1        NaN
..         ...           ...        ...     ...     ...         ...           ...        ...
6   LW8ONKO7KL      Stanford         -1    None      -1           1            -1        NaN
7   LW8ONKO7KL           UCB         -1    None      -1           1             1        NaN
8   LW8ONKO7KL    Georgetown         -1    None      -1           1             1        NaN
9   LW8ONKO7KL     NotreDame         -1    None      -1           1             1        NaN
0   YVNN6A63M8          Yale         -1    None      -1           1            -1        NaN
1   YVNN6A63M8     Dartmouth         -1    None      -1           1             1        NaN
2   YVNN6A63M8  JohnsHopkins         -1    None      -1           1            -1        NaN
0   LNBI58YD2Y         UPenn         -1    None      -1           1            -1        NaN
1   LNBI58YD2Y     Princeton         -1    None      -1           1             0        NaN
2   LNBI58YD2Y         Brown         -1    None      -1           1            -1        NaN
3   LNBI58YD2Y          Yale          1    None      -1           1             1        NaN
4   LN

In [86]:
if ('applForm' in locals()): del applForm
if ('students' in locals()): del students
applForm = ti.ApplForm()
applForm.read("collegedata_applications.csv")
students = ti.Student()
students.read("collegedata_students.csv")
students.df

studentID  classrank  admissionstest  AP  averageAP  SATsubject   GPA  GPA_w                                       program  schooltype  intendedgradyear                                            addInfo  canAfford  female  MinorityGender  MinorityRace  international  firstinfamily  sports  artist  workexp
0   6DQ8ASVPFX        NaN            1930   5   4.600000           0  3.93    NaN                        Mechanical Engineering          -1              2019  track and field 15 hours a weekVarsity letter ...          1       1              -1             1             -1            NaN      -1     NaN      NaN
0   NOP4X46KW6        NaN            2200   0        NaN           2  3.97    NaN                                       Biology          -1              2019  BoyScouts NHS NHS world language HOSA Varsity ...         -1      -1              -1             0             -1            NaN      -1     NaN      NaN
0   PF29L59LDN        NaN            2350   7   5.000000           4  4.00   4.07                                   Mathematics          -1              2019                                                            -1      -1              -1            -1             -1            NaN      -1     NaN      NaN
0   OV1DA5NRBK        NaN            2360   7   5.000000           3  4.00   4.91                        Industrial Engineering          -1              2019  Varsity Swimming 2 years 60 hours year Nationa...         -1      -1              -1            -1             -1            NaN      -1     NaN      NaN
0   NGVXH37NAJ        NaN            2250   4   4.500000           2  3.89   4.78                                           NaN           0              2019  First Cello in Youth Symphony Writing awards G...         -1       1              -1             1             -1            NaN      -1     NaN      NaN
0   G3XWLEVJ90        NaN            2170   7   4.571429           3  3.90   4.24          International Relations Biochemistry          -1              2019                                                NaN         -1      -1              -1            -1             -1            NaN      -1     NaN      NaN
0   PYCZ84NJEL        NaN            2200   0        NaN           2  4.00   4.40                             Political Science          -1              2019  Long Branch High School Junior Varsity Varsity...         -1       1              -1            -1             -1            NaN      -1     NaN      NaN
0   AZJY8J1585        NaN            2290   7   4.857143           3  4.00    NaN                                       geology          -1              2019  Employed as a church organist Select Choir Jaz...          1      -1              -1             1             -1            NaN      -1     NaN      NaN
0   VNY5CL8NZA        NaN            2110   3   4.333333           3  3.60    NaN                                    Philosophy          -1              2019  Director of African American Culture Club Gene...         -1      -1              -1             1             -1            NaN      -1     NaN      NaN
0   5KV73WIC45        NaN            2340   7   5.000000           2  3.98   4.41                                       Biology          -1              2019  Piano for 12 years scholarship pre college stu...         -1       1              -1            -1             -1            NaN      -1     NaN      NaN
0   8MRLU8M8VC        NaN            2200   2   4.500000           0  3.81   4.16                                   Engineering          -1              2019  Run two small businesses 2013 2014 FIRST robot...         -1      -1              -1             0             -1            NaN      -1     NaN      NaN
0   MA0K3TJTBY        NaN            2140   0        NaN           0  3.60   4.15                              Computer Science          -1              2019  Varsity swimming 9 12 grades 12 hours wk Robot...         -1      -1              -1      

In [83]:
if ('students' in locals()): 
    students.cleanup()
    del students
students=ti.Student()
students.read('test_student.csv')
students.df

studentID  classrank  admissionstest  AP  averageAP  SATsubject  GPA  GPA_w                          program  schooltype  intendedgradyear                                            addInfo  canAfford  female  MinorityGender  MinorityRace  international  firstinfamily  sports  artist  workexp
0  LH97PJFZWQ        NaN            2210   4          5           5    4   4.07  International Political Economy          -1              2019  National Junior Honor Society 2010 2013 Foreig...         -1       1              -1            -1             -1            NaN      -1     NaN      NaN